In [84]:
from z3 import *

# Parte 1: Construção das variáveis e SFOTS
def create_sfots(nBits, K):
    """
    Cria um SFOTS (Sistema de Transição de Estados Finito Simbolizado)
    
    Parâmetros:
    - nBits: tamanho dos BitVec
    - K: número máximo de passos
    
    Retorna: variáveis do sistema (a, b, x, y, z, loc)
    """
    # Inputs do programa
    a = BitVec('a', nBits)
    b = BitVec('b', nBits)
    
    # Variáveis de estado em cada passo
    x = [BitVec(f'x_{i}', nBits) for i in range(K + 1)]
    y = [BitVec(f'y_{i}', nBits) for i in range(K + 1)]
    z = [BitVec(f'z_{i}', nBits) for i in range(K + 1)]
    
    # Locação (nodo do CFA): 
    # 0 = estado inicial
    # 1 = loop (while)
    # 2 = erro (overflow)
    # 3 = final (y == 0)
    loc = [Int(f'loc_{i}') for i in range(K + 1)]
    
    return a, b, x, y, z, loc

def initial_state(a, b, x, y, z, loc):
    """
    Define o estado inicial do SFOTS
    
    Corresponde ao estado após:
    x, y, z = a, b, 0
    """
    return And(
        x[0] == a,      # x inicializado com a
        y[0] == b,      # y inicializado com b
        z[0] == 0,      # z inicializado com 0
        loc[0] == 0     # Estado inicial do CFA
    )

# Funções para detectar overflow nas operações de soma e shift
def detect_overflow_add(z, x, nBits):
    """
    Detecta overflow na operação z + x
    
    Para BitVec sem sinal, overflow ocorre quando:
    - O resultado é menor que qualquer dos operandos
    - Isso indica que "deu a volta" no módulo 2^nBits
    """
    result = z + x
    return Or(ULT(result, z), ULT(result, x))

def detect_overflow_shift(x, nBits):
    """
    Detecta overflow na operação x << 1
    
    Overflow ocorre se o bit mais significativo (MSB) for 1,
    pois ao fazer shift left, esse bit será perdido.
    """
    return (x & (1 << (nBits - 1))) != 0

# Função que define a relação de transição do SFOTS
def transition_relation(x, y, z, loc, step, nBits):
    """
    Define a relação de transição do SFOTS
    
    Modela o CFA com todas as transições possíveis:
    - Estado inicial → loop
    - Loop → final (quando y == 0)
    - Loop → loop (y par, sem overflow)
    - Loop → erro (y par, com overflow)
    - Loop → loop (y ímpar, sem overflow)
    - Loop → erro (y ímpar, com overflow)
    - Erro → erro (permanece)
    - Final → final (permanece)
    """
    even = (y[step] & 1) == 0  # Verifica se y é par
    
    # Transição 1: Estado inicial → loop
    trans_init = Implies(
        loc[step] == 0,
        And(
            loc[step + 1] == 1,
            x[step + 1] == x[step],
            y[step + 1] == y[step],
            z[step + 1] == z[step]
        )
    )
    
    # Transição 2: Loop termina quando y == 0
    trans_loop_end = Implies(
        And(loc[step] == 1, y[step] == 0),
        And(
            loc[step + 1] == 3,  # Vai para estado final
            x[step + 1] == x[step],
            y[step + 1] == y[step],
            z[step + 1] == z[step]
        )
    )
    
    # Transição 3: y é par e SEM overflow
    trans_loop_even = Implies(
        And(
            loc[step] == 1,
            y[step] != 0,
            even,
            Not(detect_overflow_shift(x[step], nBits))
        ),
        And(
            loc[step + 1] == 1,  # Continua no loop
            x[step + 1] == (x[step] << 1),      # Duplica x
            y[step + 1] == LShR(y[step], 1),    # Divide y por 2 (shift lógico)
            z[step + 1] == z[step]              # z não muda
        )
    )
    
    # Transição 4: y é par mas COM overflow no shift
    trans_loop_even_error = Implies(
        And(
            loc[step] == 1,
            y[step] != 0,
            even,
            detect_overflow_shift(x[step], nBits)
        ),
        And(
            loc[step + 1] == 2,  # Vai para estado de erro
            x[step + 1] == x[step],
            y[step + 1] == y[step],
            z[step + 1] == z[step]
        )
    )
    
    # Transição 5: y é ímpar e SEM overflow
    trans_loop_odd = Implies(
        And(
            loc[step] == 1,
            y[step] != 0,
            Not(even),
            Not(detect_overflow_add(z[step], x[step], nBits))
        ),
        And(
            loc[step + 1] == 1,  # Continua no loop
            x[step + 1] == x[step],              # x não muda
            y[step + 1] == y[step] - 1,          # Decrementa y
            z[step + 1] == z[step] + x[step]     # Adiciona x a z
        )
    )
    
    # Transição 6: y é ímpar mas COM overflow na soma
    trans_loop_odd_error = Implies(
        And(
            loc[step] == 1,
            y[step] != 0,
            Not(even),
            detect_overflow_add(z[step], x[step], nBits)
        ),
        And(
            loc[step + 1] == 2,  # Vai para estado de erro
            x[step + 1] == x[step],
            y[step + 1] == y[step],
            z[step + 1] == z[step]
        )
    )
    
    # Transição 7: Estado de erro permanece em erro
    trans_error = Implies(
        loc[step] == 2,
        And(
            loc[step + 1] == 2,
            x[step + 1] == x[step],
            y[step + 1] == y[step],
            z[step + 1] == z[step]
        )
    )
    
    # Transição 8: Estado final permanece final
    trans_final = Implies(
        loc[step] == 3,
        And(
            loc[step + 1] == 3,
            x[step + 1] == x[step],
            y[step + 1] == y[step],
            z[step + 1] == z[step]
        )
    )
    
    return And(
        trans_init,
        trans_loop_end,
        trans_loop_even,
        trans_loop_even_error,
        trans_loop_odd,
        trans_loop_odd_error,
        trans_error,
        trans_final
    )


In [85]:
def verify_invariant(nBits, K, verbose=True):
    """
    Verifica se x*y + z == a*b é um invariante do sistema.
    """
    if verbose:
        print(f"\n{'='*60}")
        print(f"QUESTÃO 2: Verificação do Invariante")
        print(f"{'='*60}")
        print(f"Configuração:")
        print(f"  - Bits: {nBits}")
        print(f"  - Passos (K): {K}")
        print(f"  - Invariante: x*y + z == a*b")
        print(f"\nIniciando verificação...\n")
    
    a, b, x, y, z, loc = create_sfots(nBits, K)
    solver = Solver()
    solver.set("timeout", 500000)  # Timeout reduzido de 30s para 5s
    
    solver.add(initial_state(a, b, x, y, z, loc))
    solver.add(UGE(a, 0))  
    solver.add(UGE(b, 0))  
    
    if verbose:
        print("✓ Estado inicial configurado")
    
    for step in range(K):
        solver.add(transition_relation(x, y, z, loc, step, nBits))
    
    invariant_violations = []
    for step in range(K + 1):
        violation = And(
            loc[step] != 2,  # Não é estado de erro
            x[step] * y[step] + z[step] != a * b  # Invariante violado
        )
        invariant_violations.append(violation)
    
    solver.add(Or(invariant_violations))
    
    if verbose:
        print(f"\n{'─'*60}")
        print("Procurando por violações do invariante...")
        print(f"{'─'*60}")
    
    result = solver.check()
    
    if result == unknown:
        if verbose:
            print("\n⚠ TIMEOUT: Verificação demorou muito tempo")
        return None
    
    if result == unsat:
        if verbose:
            print("\n" + "="*60)
            print("✓ RESULTADO: O invariante é VÁLIDO!")
            print("="*60)
        return True
    
    else:
        if verbose:
            print("\n" + "="*60)
            print("✗ RESULTADO: O invariante foi VIOLADO!")
            print("="*60)
        return False


In [86]:
def verify_safety(nBits, K, N, M, verbose=True):
    """
    Verifica se o estado de erro é inacessível com as restrições
    N < a, b < M.
    """
    if verbose:
        print(f"\n{'='*60}")
        print(f"QUESTÃO 3: Verificação de Segurança")
        print(f"{'='*60}")
        print(f"Configuração:")
        print(f"  - Bits: {nBits}")
        print(f"  - Passos (K): {K}")
        print(f"  - Restrições: {N} < a, b < {M}")
        print(f"  - Valores máximos representáveis: 0 a {2**nBits - 1}")
        print(f"\nIniciando verificação...\n")
    
    # Criar o SFOTS
    a, b, x, y, z, loc = create_sfots(nBits, K)
    
    # Criar o solver Z3 com timeout
    solver = Solver()
    solver.set("timeout", 500000)  # 5 segundos de timeout para testes
    
    # Adicionar estado inicial
    solver.add(initial_state(a, b, x, y, z, loc))
    
    # ADICIONAR RESTRIÇÕES: N < a, b < M
    solver.add(UGT(a, N))  # a > N (unsigned greater than)
    solver.add(ULT(a, M))  # a < M (unsigned less than)
    solver.add(UGT(b, N))  # b > N
    solver.add(ULT(b, M))  # b < M
    
    if verbose:
        print("✓ Estado inicial configurado")
        print(f"✓ Restrições adicionadas:")
        print(f"  - {N} < a < {M}")
        print(f"  - {N} < b < {M}")
    
    # Adicionar todas as transições
    for step in range(K):
        solver.add(transition_relation(x, y, z, loc, step, nBits))
    
    if verbose:
        print(f"✓ Relações de transição adicionadas ({K} passos)")
    
    # VERIFICAÇÃO DE SEGURANÇA:
    # Procuramos por um estado de erro (loc == 2)
    # Se não encontrarmos, o programa é seguro!
    
    error_states = []
    for step in range(K + 1):
        error_states.append(loc[step] == 2)
    
    # Adiciona: "existe algum passo onde chegamos ao erro?"
    solver.add(Or(error_states))
    
    if verbose:
        print(f"\n{'─'*60}")
        print("Procurando por caminhos que levam ao estado de erro...")
        print(f"{'─'*60}")
    
    # Verificar satisfatibilidade
    result = solver.check()
    
    if result == unknown:
        if verbose:
            print("\n⚠ TIMEOUT: Verificação demorou muito tempo")
        return None
    
    if result == unsat:
        # UNSAT significa que NÃO existe caminho para o erro
        # Logo, o programa é SEGURO!
        if verbose:
            print("\n" + "="*60)
            print("✓ RESULTADO: O programa é SEGURO!")
            print("="*60)
            print(f"\nCom as restrições {N} < a, b < {M}:")
            print(f"  ✓ O estado de erro NÃO é acessível")
            print(f"  ✓ Nenhum overflow pode ocorrer em {K} passos")
            print(f"  ✓ O programa termina corretamente para todos os inputs")
            print(f"\nInterpretação:")
            print(f"  - Para valores de a e b neste intervalo,")
            print(f"    o resultado a*b cabe em {nBits} bits")
            print(f"  - Não há overflow nas operações intermediárias")
        return True
    
    else:
        # SAT significa que encontramos um caminho para o erro
        # O programa NÃO é seguro com estas restrições
        if verbose:
            print("\n" + "="*60)
            print("✗ RESULTADO: O programa NÃO é seguro!")
            print("="*60)
            
            model = solver.model()
            a_val = model[a].as_long()
            b_val = model[b].as_long()
            
            print(f"\nContra-exemplo encontrado:")
            print(f"  a = {a_val}")
            print(f"  b = {b_val}")
            print(f"  a * b = {a_val * b_val}")
            print(f"  Máximo representável em {nBits} bits: {2**nBits - 1}")
            
            if a_val * b_val > 2**nBits - 1:
                print(f"  ⚠ Overflow! {a_val * b_val} > {2**nBits - 1}")
            
            # Encontrar o passo onde ocorre o erro
            for step in range(K + 1):
                if model.evaluate(loc[step]) == 2:
                    print(f"\n  Estado de erro alcançado no passo {step}:")
                    print(f"    x[{step}] = {model[x[step]]}")
                    print(f"    y[{step}] = {model[y[step]]}")
                    print(f"    z[{step}] = {model[z[step]]}")
                    
                    # Identificar qual operação causou o overflow
                    if step > 0:
                        prev_loc = model.evaluate(loc[step-1])
                        prev_y = model[y[step-1]].as_long()
                        prev_x = model[x[step-1]].as_long()
                        prev_z = model[z[step-1]].as_long()
                        
                        print(f"\n  Estado anterior (passo {step-1}):")
                        print(f"    x[{step-1}] = {prev_x}")
                        print(f"    y[{step-1}] = {prev_y}")
                        print(f"    z[{step-1}] = {prev_z}")
                        
                        if prev_y % 2 == 0:
                            print(f"\n  Overflow na operação: x << 1")
                            print(f"    Tentou calcular: {prev_x} << 1 = {prev_x * 2}")
                            print(f"    Máximo permitido: {2**nBits - 1}")
                        else:
                            print(f"\n  Overflow na operação: z + x")
                            print(f"    Tentou calcular: {prev_z} + {prev_x} = {prev_z + prev_x}")
                            print(f"    Máximo permitido: {2**nBits - 1}")
                    
                    break
        return False


In [ ]:
if __name__ == "__main__":
    print("="*60)
    print("TESTE COMPLETO - VERIFICAÇÃO DO INVARIANTE E SEGURANÇA")
    print("="*60)
    
    # Teste da Parte 1 (Construção das variáveis e SFOTS)
    print("\n### TESTE 1: Construção de Variáveis e SFOTS ###")
    nBits = 8
    K = 10  # Número de passos para teste
    a, b, x, y, z, loc = create_sfots(nBits, K)
    print(f"✓ Variáveis criadas para {nBits} bits e {K} passos.")
    
    # Teste da Parte 2 (Verificação do Invariante)
    print("\n### TESTE 2: Verificação do Invariante ###")
    invar_result = verify_invariant(nBits=nBits, K=K, verbose=True)
    
    if invar_result:
        print("✓ O invariante x*y + z = a*b foi validado com sucesso.")
    else:
        print("✗ O invariante x*y + z = a*b foi violado.")
    
    # Teste da Parte 3 (Verificação de Segurança)
    print("\n### TESTE 3: Verificação de Segurança ###")
    N, M = 0, 10  # Restrições de segurança (pode-se ajustar)
    safety_result = verify_safety(nBits=nBits, K=K, N=N, M=M, verbose=True)
    
    if safety_result:
        print(f"✓ O programa é seguro. O estado de erro NÃO é acessível com {N} < a, b < {M}.")
    else:
        print(f"✗ O programa NÃO é seguro. Encontrado caminho para erro com {N} < a, b < {M}.")
    
    print("\n" + "="*60)
    print("TESTE COMPLETO FINALIZADO!")
    print("="*60)


TESTE COMPLETO - VERIFICAÇÃO DO INVARIANTE E SEGURANÇA

### TESTE 1: Construção de Variáveis e SFOTS ###
✓ Variáveis criadas para 8 bits e 10 passos.

### TESTE 2: Verificação do Invariante ###

QUESTÃO 2: Verificação do Invariante
Configuração:
  - Bits: 8
  - Passos (K): 10
  - Invariante: x*y + z == a*b

Iniciando verificação...

✓ Estado inicial configurado

────────────────────────────────────────────────────────────
Procurando por violações do invariante...
────────────────────────────────────────────────────────────

✓ RESULTADO: O invariante é VÁLIDO!
✓ O invariante x*y + z = a*b foi validado com sucesso.

### TESTE 3: Verificação de Segurança ###

QUESTÃO 3: Verificação de Segurança
Configuração:
  - Bits: 8
  - Passos (K): 10
  - Restrições: 0 < a, b < 10
  - Valores máximos representáveis: 0 a 255

Iniciando verificação...

✓ Estado inicial configurado
✓ Restrições adicionadas:
  - 0 < a < 10
  - 0 < b < 10
✓ Relações de transição adicionadas (10 passos)

───────────────────

In [88]:


print("\n" + "="*70)
print("TESTES ADICIONAIS")
print("="*70)

# ============================================================
# TESTE 4: Verificar limites de segurança - Valores médios
# ============================================================
print("\n### TESTE 4: Valores Médios (0 < a,b < 16) ###")
print("Expectativa: SEGURO (16 x 16 = 256, mas pode dar overflow nas operações)")
result4 = verify_safety(nBits=8, K=15, N=0, M=16, verbose=True)

# ============================================================
# TESTE 5: Valores grandes - Esperado OVERFLOW
# ============================================================
print("\n" + "-"*70)
print("\n### TESTE 5: Valores Grandes (100 < a,b < 150) ###")
print("Expectativa: INSEGURO (overflow garantido)")
result5 = verify_safety(nBits=8, K=20, N=100, M=150, verbose=True)

# ============================================================
# TESTE 6: Limite exato de 8 bits
# ============================================================
print("\n" + "-"*70)
print("\n### TESTE 6: No limite de 8 bits (0 < a,b < 16) ###")
print("Máximo representável em 8 bits: 255")
print("16 x 16 = 256 > 255 (overflow!)")
result6 = verify_safety(nBits=8, K=15, N=0, M=16, verbose=False)
if result6:
    print("✓ SEGURO: Não encontrou overflow")
elif result6 is None:
    print("⚠ TIMEOUT")
else:
    print("✗ INSEGURO: Encontrou overflow")

# ============================================================
# TESTE 7: Testar com 16 bits e valores maiores
# ============================================================
print("\n" + "-"*70)
print("\n### TESTE 7: 16 bits com valores maiores (0 < a,b < 200) ###")
print("Expectativa: SEGURO (200 x 200 = 40000 < 65535)")
result7 = verify_safety(nBits=16, K=20, N=0, M=200, verbose=False)
if result7:
    print("✓ SEGURO: Programa funciona corretamente")
    print("  200 x 200 = 40000 < 65535 (máximo em 16 bits)")
elif result7 is None:
    print("⚠ TIMEOUT")
else:
    print("✗ INSEGURO: Encontrou overflow")

# ============================================================
# TESTE 8: 16 bits próximo do limite
# ============================================================
print("\n" + "-"*70)
print("\n### TESTE 8: 16 bits no limite (200 < a,b < 300) ###")
print("Máximo representável em 16 bits: 65535")
print("300 x 300 = 90000 > 65535 (overflow!)")
result8 = verify_safety(nBits=16, K=25, N=200, M=300, verbose=False)
if result8:
    print("✓ SEGURO: Não encontrou overflow")
elif result8 is None:
    print("⚠ TIMEOUT")
else:
    print("✗ INSEGURO: Encontrou overflow como esperado")

# ============================================================
# TESTE 9: Invariante com mais passos
# ============================================================
print("\n" + "-"*70)
print("\n### TESTE 9: Invariante com mais passos (K=20) ###")
result9 = verify_invariant(nBits=8, K=20, verbose=False)
if result9:
    print("✓ INVARIANTE VÁLIDO com 20 passos")
elif result9 is None:
    print("⚠ TIMEOUT")
else:
    print("✗ INVARIANTE VIOLADO")

# ============================================================
# TESTE 10: Caso extremo - valores muito pequenos
# ============================================================
print("\n" + "-"*70)
print("\n### TESTE 10: Valores Muito Pequenos (0 < a,b < 3) ###")
print("Expectativa: SEGURO (3 x 3 = 9)")
result10 = verify_safety(nBits=8, K=10, N=0, M=3, verbose=False)
if result10:
    print("✓ SEGURO: Valores muito pequenos funcionam corretamente")
elif result10 is None:
    print("⚠ TIMEOUT")
else:
    print("✗ INSEGURO (inesperado!)")

# ============================================================
# TESTE 11: Testar com 4 bits (muito restritivo)
# ============================================================
print("\n" + "-"*70)
print("\n### TESTE 11: 4 bits com valores pequenos (0 < a,b < 4) ###")
print("Máximo representável em 4 bits: 15")
print("4 x 4 = 16 > 15 (overflow!)")
result11 = verify_safety(nBits=4, K=10, N=0, M=4, verbose=False)
if result11:
    print("✓ SEGURO")
elif result11 is None:
    print("⚠ TIMEOUT")
else:
    print("✗ INSEGURO: Encontrou overflow em 4 bits")

# ============================================================
# TESTE 12: Intervalo específico que deve ser seguro
# ============================================================
print("\n" + "-"*70)
print("\n### TESTE 12: Intervalo Seguro Calculado (0 < a,b < 15) ###")
print("15 x 15 = 225 < 255 ✓")
result12 = verify_safety(nBits=8, K=15, N=0, M=15, verbose=False)
if result12:
    print("✓ SEGURO: Confirmado que 15x15=225 cabe em 8 bits")
elif result12 is None:
    print("⚠ TIMEOUT")
else:
    print("✗ INSEGURO: Overflow nas operações intermediárias!")

# ============================================================
# SUMÁRIO DOS TESTES
# ============================================================
print("\n" + "="*70)
print("SUMÁRIO DOS TESTES")
print("="*70)

tests_results = [
    ("Teste 4: 8 bits, 0<a,b<16", result4),
    ("Teste 5: 8 bits, 100<a,b<150 (overflow esperado)", result5),
    ("Teste 6: 8 bits, limite exato", result6),
    ("Teste 7: 16 bits, 0<a,b<200", result7),
    ("Teste 8: 16 bits, 200<a,b<300 (overflow esperado)", result8),
    ("Teste 9: Invariante K=20", result9),
    ("Teste 10: 8 bits, 0<a,b<3", result10),
    ("Teste 11: 4 bits, 0<a,b<4", result11),
    ("Teste 12: 8 bits, 0<a,b<15", result12),
]

print(f"\n{'Teste':<50} {'Resultado':<20}")
print("-"*70)

for test_name, result in tests_results:
    if result is True:
        status = "✓ SEGURO/VÁLIDO"
    elif result is False:
        status = "✗ INSEGURO/VIOLADO"
    else:
        status = "⚠ TIMEOUT"
    print(f"{test_name:<50} {status:<20}")

# ============================================================
# ANÁLISE DE LIMITES TEÓRICOS
# ============================================================
print("\n" + "="*70)
print("ANÁLISE TEÓRICA DOS LIMITES")
print("="*70)

def analyze_limits(nBits):
    max_val = 2**nBits - 1
    safe_sqrt = int(max_val ** 0.5)
    
    print(f"\n### {nBits} bits ###")
    print(f"  Máximo representável: {max_val}")
    print(f"  Raiz quadrada: {safe_sqrt}")
    print(f"  Limite seguro teórico: a, b ≤ {safe_sqrt}")
    print(f"  Verificação: {safe_sqrt} × {safe_sqrt} = {safe_sqrt * safe_sqrt}")
    
    if safe_sqrt * safe_sqrt <= max_val:
        print(f"  ✓ {safe_sqrt}² = {safe_sqrt * safe_sqrt} ≤ {max_val}")
    
    print(f"\n  Exemplos de limites:")
    for limit in [2, 4, 8, 16, 32, 64, 128]:
        if limit <= safe_sqrt:
            produto = limit * limit
            status = "✓" if produto <= max_val else "✗"
            print(f"    {status} a,b < {limit}: máx produto = {produto}")

analyze_limits(4)
analyze_limits(8)
analyze_limits(16)
analyze_limits(32)

print("\n" + "="*70)
print("TESTES ADICIONAIS COMPLETOS!")
print("="*70)


TESTES ADICIONAIS

### TESTE 4: Valores Médios (0 < a,b < 16) ###
Expectativa: SEGURO (16 x 16 = 256, mas pode dar overflow nas operações)

QUESTÃO 3: Verificação de Segurança
Configuração:
  - Bits: 8
  - Passos (K): 15
  - Restrições: 0 < a, b < 16
  - Valores máximos representáveis: 0 a 255

Iniciando verificação...



✓ Estado inicial configurado
✓ Restrições adicionadas:
  - 0 < a < 16
  - 0 < b < 16
✓ Relações de transição adicionadas (15 passos)

────────────────────────────────────────────────────────────
Procurando por caminhos que levam ao estado de erro...
────────────────────────────────────────────────────────────

✓ RESULTADO: O programa é SEGURO!

Com as restrições 0 < a, b < 16:
  ✓ O estado de erro NÃO é acessível
  ✓ Nenhum overflow pode ocorrer em 15 passos
  ✓ O programa termina corretamente para todos os inputs

Interpretação:
  - Para valores de a e b neste intervalo,
    o resultado a*b cabe em 8 bits
  - Não há overflow nas operações intermediárias

----------------------------------------------------------------------

### TESTE 5: Valores Grandes (100 < a,b < 150) ###
Expectativa: INSEGURO (overflow garantido)

QUESTÃO 3: Verificação de Segurança
Configuração:
  - Bits: 8
  - Passos (K): 20
  - Restrições: 100 < a, b < 150
  - Valores máximos representáveis: 0 a 255

Iniciando 